<a href="https://colab.research.google.com/github/dspraneeth07/AXIOM--SQL-REFLEX-AGENT-v4/blob/main/notebooks/00_env_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q duckdb pandas numpy networkx faiss-cpu sentence-transformers tqdm
!pip install -q gdown


In [ ]:
import duckdb
import pandas as pd
import numpy as np
import networkx as nx

print("Environment ready ✅")


Environment ready ✅


In [ ]:
import os

folders = [
    "data",
    "data/spider",
    "data/bird",
    "schemas",
    "agents",
    "outputs"
]

for f in folders:
    os.makedirs(f, exist_ok=True)

print("Project folders created ✅")


Project folders created ✅


In [ ]:
!mkdir -p data/spider
!wget -q https://raw.githubusercontent.com/taoyds/spider/master/evaluation_examples/examples/dev.json -O data/spider/dev.json
!wget -q https://raw.githubusercontent.com/taoyds/spider/master/evaluation_examples/examples/train_spider.json -O data/spider/train_spider.json
!wget -q https://raw.githubusercontent.com/taoyds/spider/master/evaluation_examples/examples/tables.json -O data/spider/tables.json


In [ ]:
!mkdir -p data
!gdown --id 1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J -O spider.zip


/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J
From (redirected): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J&confirm=t&uuid=4c902584-ffdb-460a-9d08-444141021595
To: /content/spider.zip
100% 206M/206M [00:03<00:00, 64.9MB/s]


In [ ]:
!unzip -q spider.zip -d data/spider


In [ ]:
import os
os.listdir("data/spider")


['spider_data', '__MACOSX', 'dev.json', 'train_spider.json', 'tables.json']

In [ ]:
!git clone https://github.com/AlibabaResearch/DAMO-ConvAI.git data/bird_raw


Cloning into 'data/bird_raw'...
remote: Enumerating objects: 5768, done.
remote: Counting objects: 100% (487/487), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 5768 (delta 309), reused 305 (delta 265), pack-reused 5281 (from 2)
Receiving objects: 100% (5768/5768), 168.81 MiB | 16.06 MiB/s, done.
Resolving deltas: 100% (1916/1916), done.
Updating files: 100% (4894/4894), done.


In [ ]:
os.listdir("data/bird_raw")


['SDPO',
 'unid2t',
 'FlowBench',
 'space-3',
 'EPO',
 's2sql',
 'sunsql',
 'proton',
 'cgodial',
 'LICENSE',
 'star',
 'api-bank',
 'dstc11-simmc',
 'bird',
 '.gitignore',
 'dial2vec',
 'dialogue-cse',
 'mmevol',
 'r2sql',
 'doc2bot',
 'metaretriever',
 'diana',
 'tkk',
 'OpenOmni',
 'dater',
 'dial-start',
 'oltqa',
 'acl23doc2dial',
 'WideDeep',
 'spokenwoz',
 '.github',
 'PRO',
 'rlp',
 'OmniCharacter',
 'pcll',
 'MaskedThought',
 'space-1',
 'README.md',
 'ssll',
 'Loong',
 '.git',
 'IOPO',
 'space-2',
 '.DS_Store',
 'sdsql',
 'deep-thinking',
 'graphix',
 'cats',
 'pace',
 'Awesome-Self-Evolution-of-LLM',
 'attention-buckets',
 'spectra']

In [ ]:
import duckdb

db = duckdb.connect("axiom_v4.duckdb")
print("DuckDB initialized ✅")


DuckDB initialized ✅


In [ ]:
import sqlite3

sqlite_db = "data/spider/spider_data/database/academic/academic.sqlite"
sqlite_conn = sqlite3.connect(sqlite_db)

tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table'",
    sqlite_conn
)
tables


,name
0,author
1,conference
2,domain
3,domain_author
4,domain_conference
5,journal
6,domain_journal
7,keyword
8,domain_keyword
9,publication


In [ ]:
table_name = tables.iloc[0]["name"]
df = pd.read_sql(f"SELECT * FROM {table_name} LIMIT 100", sqlite_conn)

db.execute(f"CREATE OR REPLACE TABLE academic_{table_name} AS SELECT * FROM df")
print("Spider DB loaded into DuckDB ✅")


Spider DB loaded into DuckDB ✅


In [ ]:
db.execute("SHOW TABLES").fetchdf()


,name
0,academic_author


In [ ]:
db.execute("SELECT COUNT(*) FROM academic_" + table_name).fetchone()


(0,)

DATA CONTRACTS (v4)

SchemaGraph:
{
  "tables": [
    {
      "table_name": str,
      "columns": [str],
      "primary_key": [str],
      "foreign_keys": [
        {
          "column": str,
          "ref_table": str,
          "ref_column": str
        }
      ]
    }
  ]
}

AgentInput:
{
  "question": str,
  "candidate_tables": [str],
  "schema_graph": object
}

AgentOutput:
{
  "sql": str,
  "confidence": float,
  "reason": str
}


In [20]:
!pwd
!ls


/content/agents/AXIOM--SQL-REFLEX-AGENT-v4
00_env_setup.ipynb     data	notebooks  README.md
01_schema_graph.ipynb  LICENSE	outputs


In [21]:
!cp -r /content/* .


cp: cannot copy a directory, '/content/agents', into itself, './agents'


In [22]:
!ls


00_env_setup.ipynb     AXIOM--SQL-REFLEX-AGENT-v4  LICENSE    README.md
01_schema_graph.ipynb  axiom_v4.duckdb		   notebooks  schemas
agents		       data			   outputs    spider.zip


In [23]:
!git add .
!git commit -m "FULL DUMP: complete Colab state (data + outputs + everything)"
!git push


hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> AXIOM--SQL-REFLEX-AGENT-v4
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached AXIOM--SQL-REFLEX-AGENT-v4
hint: 
hint: See "git help submodule" for more information.
[main cb86792] FULL DUMP: complete Colab state (data + outputs + everything)
 2248 files changed, 5909944 insertions(+)
 create mode 160000 AXIOM--SQL-REFLEX-AGENT-v4
 create mode 160000 agents/AXIOM--SQL-REFLEX-AGENT-v4
 create mode 100644 axiom_v4.duckdb
 create mode 160000 data/bird_raw
 create mode 100644 data/spider/__MACOSX/spider_data/._.DS_Store
 create mode 100644 data/spider/__MACOSX/spider_data/._README.txt
 create mode 100755 data/spider/__MACOSX/spider_d